# Tutorial 1: Loading and Manipulating Data in Pandas

The goal of this tutorial is to get you comfortable doing data analysis using the Pandas library. Pandas is a very powerful library for manipulating data. The documentation can be found online [here](http://pandas.pydata.org/pandas-docs/stable/). You'll want to get comfortable with it. The tutorial assumes you are comfortable with basic constructs from computer programming such as variables, lists, and conditional statements. 

In this tutorial you'll learn how to:
- Load data from .csv files. 
- Manipulate tabular data in a dataframe to select, filter, transform, sort, and aggregate it. 

### Finding Data
Ok, let's get some data first, then we'll figure out how to load it. Every year the TSA (Transportation Security Agency) confiscates thousands of weapons at airports. Here's a [dataset](https://github.com/comp-journalism/UMD-J479V-J779V-Spring2017/raw/master/Data/tsa-dangerous-items-fy15.csv) of the more than 22,000 weapons that were confiscated in 2015. Right-click that link in the last sentence and download the .csv file locally to your computer. 

### Loading Data
In order to use the Pandas library we need to import it so that Python knows how to access the functionality that it provides. The convention is to call it `pd` as in the following line of code:

In [ ]:
import pandas as pd

There are a variety of 'magic' functions that can be useful in Jupyter Notebooks. These are prefixed by the '%' character. 

Here we're interested to know the current directory of the .ipynb file (this file) so that we can load another file using a relative path. We can get this by using the `%pwd` magic function. (BTW, to see an entire listing of available magic functions use `%quickref`)


In [ ]:
%pwd

Now that we know the current directory of this script, we can construct the correct relative path to our data file. You can use the `read_csv` function and pass the relative path to the file in order to load it. There are *a lot* of [other parameters](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html#pandas.read_csv), so be sure to get comfortable looking at the documentation to understand all of the options. 

In [ ]:
pd.read_csv("Data/tsa-dangerous-items-fy15.csv")

We can easily assign the read data frame to a variable that we can then manipulate in different ways. Writing the variable as the last line in a Jupyter cell will print it for you. 

In [ ]:
tsa_df = pd.read_csv("Data/tsa-dangerous-items-fy15.csv")
tsa_df

Notice how it has numbered the rows in the first column on the left (that is called the index column) and how the first row is a header row with the names of the columns. If you just want to know the names of the columns you can:

In [ ]:
tsa_df.columns

Let's also parse the date column from strings into actual date objects. We can do this by passing an extra parameter to the `read_csv` function that tells it which columns to treat as dates. This is useful if we want to manipulate or filter by dates later on. 

In [ ]:
tsa_df = pd.read_csv("Data/tsa-dangerous-items-fy15.csv", parse_dates = ["date"])
tsa_df

Note in the output the dates now follow a standard formatting of YYYY-MM-DD when printed, and internally they're stored as python [datetime objects](https://docs.python.org/2/library/datetime.html). 

### Saving Data
You can output a dataframe to a .csv file after you've manipulated it. We use a function called `to_csv` and a parameter to strip the index column back out. 


In [ ]:
tsa_df.to_csv("Data/test_output.csv", index=False)

### Manipulating Data in Pandas
There is a LOT that a DataFrame can do. You can familiarize yourself with all it offers in the [documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). 

To select a single column of data from the DataFrame you have a couple options:
- Use the name of the column within brackets like this: *dataframe["column_name"]*
- Use the name of the column after a period like this: *dataframe.column_name*


In [ ]:
tsa_df["item"]

In [ ]:
tsa_df.item

You can get multiple columns by specifying them in a list.


In [ ]:
tsa_df[["item", "date"]]

And if we want to get just one row of data across columns we can use a function called `iloc` to extract that row based on the row index. There are other ways to get at data which you can read about in the [indexing documentation](http://pandas.pydata.org/pandas-docs/stable/indexing.html)

In [ ]:
tsa_df[["item", "date"]].iloc[3]

You can access the row as a python array of data by using the `values` property. 

In [ ]:
tsa_df[["item", "date"]].iloc[3].values

To access only a certain range of items from the column (called a [slice](http://pandas.pydata.org/pandas-docs/stable/indexing.html#slicing-ranges)) you can use brackets and specify the range of items

In [ ]:
print tsa_df.item[0:5] # slices the first 5 objects (starting from the first index which is zero)
print tsa_df.item[-1:-6:-1] # slices the last 5 objects

### Filtering
You can filter based on data values using conditionals on the columns. For instance, to filter for just the items that were "Firearms" we can do something like this:

In [ ]:
tsa_firearms_df = tsa_df[tsa_df.item == "Firearms"]
tsa_firearms_df

After all that filtering you might wonder how much data you have left. To check the shape (i.e. number of rows and columns) of a DataFrame just append ``.shape`` at the end. 

In [ ]:
print tsa_df.shape
print tsa_firearms_df.shape

Or maybe we want the items that were Firearms that were confiscated at Baltimore airport, BWI. We can do this by putting each condition in parentheses and using the boolean AND operator which is the ampersand (i.e., "&"). You could also combine different filtering criteria using an OR operator which is the pipe (i.e., "|").

In [ ]:
bwi_firearms_df = tsa_df[(tsa_df.item == "Firearms") & (tsa_df.airport_code == "BWI")]
bwi_firearms_df

You'll notice that in the filtered data frame the index starts from a different number "1035", but maybe we want to reset it to start at zero now that we're focused on BWI. We can do that using the `reset_index` function. The `drop = True` parameter tells it to reset the index to the default integer index. 

In [ ]:
bwi_firearms_df = bwi_firearms_df.reset_index(drop=True)
bwi_firearms_df

#### Removing missing data

If you scroll to the bottom of the original dataframe you will see that there are three rows that are listed "Nan". These are empty rows and we want to filter them out. You notice that a lot of datasets you work with are deficient in some way or another. For instance, they may be missing values in some rows and columns. When it loads a file in Pandas is smart enough to mark empty fields as "NaN" which stands for Not a Number. 

We can test for these values using the ``isnull`` and ``notnull`` functions which will return a True / False value based on the value of the item. 

In [ ]:
tsa_df.item.isnull()

And we may want to filter out rows with those empty values. We can do that with a special selector syntax. In the following notice that within the brackets we tell it to select rows for which type_telemarketing is not null. Another useful function for removing missing data is `dropna()` which has [parameters](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.dropna.html) that allow you to drop rows or columns have have any or all values that are missing. 

In [ ]:
tsa_df[tsa_df.item.notnull()]

In [ ]:
# Another way to drop the rows with missing data
tsa_df.dropna()

In [ ]:
tsa_df = tsa_df.dropna()

### Sorting
Oftentimes you will want to sort your data to get an overview or see what is at the top or bottom of a ranking. To sort by values use the `sort_values` [function](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values). We can sort our BWI data by the date for instance:

In [ ]:
bwi_firearms_sorted_df = bwi_firearms_df.sort_values(by="date", ascending=False)
bwi_firearms_sorted_df

### Aggregation
You'll often want to summarize DataFrames to get an overview of your data, or to aggregate it. The `describe()` function is useful for an initial overview, but there are many others such as `min()`, `max()`, `sum()`, `mean()`, and [many others](http://pandas.pydata.org/pandas-docs/stable/basics.html#descriptive-statistics)

In [ ]:
tsa_df.describe()

A useful analytic operation is to create groups that can then be summarized or filtered. This can be accomplished with the `groupby()` [function](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html). Various [aggregation functions](http://pandas.pydata.org/pandas-docs/stable/groupby.html) can then be applied. 

In [ ]:
grouped = tsa_df.groupby("airport_code")
grouped.size()

You can get a single group.

In [ ]:
grouped.get_group("BWI")

You can also filter groups based on different aggregate values, such as the size of the group. 

In [ ]:
# Filter on airports with more than 500 confiscations, note after the filter we then have to regroup. 
grouped.filter(lambda x: len(x) > 500).groupby("airport_code").size()

### Transformation
One of the problems with grabbing the "BWI" group above is that it misses some data that was mistakenly input as lowercase "bwi" (scroll to the bottom of the cell that lists the size of groups). To normalize that field of the data we need to transform it to be all uppercase. 

Sometimes you will want to transform your data by applying a transformation function to each datum within a column or row. We define a function which takes in an input datum (x in this case) and returns the transformed value of that. We use the `apply` [function](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.apply.html) on the dataframe to apply that function to an entire column (or to an entire row).

In [ ]:
def uppercaser(x):
    return x.upper()

tsa_df.airport_code.apply(uppercaser)

In [ ]:
grouped = tsa_df.groupby("airport_code")
grouped.get_group("BWI").shape